## Import

In [1]:
import os
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt
import torch

from transformers import VisionEncoderDecoderModel, ViTImageProcessor, AutoTokenizer
from transformers import BlipProcessor, BlipForConditionalGeneration

from PIL import Image, ImageFile


ImageFile.LOAD_TRUNCATED_IMAGES = True

2024-06-12 22:09:27.863884: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-06-12 22:09:27.864031: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-06-12 22:09:27.990948: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


## Data

In [2]:
image_folder_path = "/kaggle/input/memotion-dataset-7k/memotion_dataset_7k/images"
image_paths = []
for filename in os.listdir(image_folder_path):
    image_paths.append(f"{image_folder_path}/{filename}")
print(len(image_paths))

6992


In [3]:
image_paths[0]

'/kaggle/input/memotion-dataset-7k/memotion_dataset_7k/images/image_5576.jpg'

## GPT2 image captioning

In [4]:
#model = VisionEncoderDecoderModel.from_pretrained("nlpconnect/vit-gpt2-image-captioning")
#feature_extractor = ViTImageProcessor.from_pretrained("nlpconnect/vit-gpt2-image-captioning")
#tokenizer = AutoTokenizer.from_pretrained("nlpconnect/vit-gpt2-image-captioning")

#device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
#model.to(device);

In [5]:
#max_length = 128
#num_beams = 4
#gen_kwargs = {"max_length": max_length, "num_beams": num_beams}

#def predict_step(image_paths):
#    images = []
#    for image_path in image_paths:
#        #print(image_path)
#        i_image = Image.open(image_path)
#        if i_image.mode != "RGB":
#            i_image = i_image.convert(mode="RGB")
#        images.append(i_image)

#    pixel_values = feature_extractor(images=images, return_tensors="pt").pixel_values
#    pixel_values = pixel_values.to(device)

#    output_ids = model.generate(pixel_values, **gen_kwargs)

#    preds = tokenizer.batch_decode(output_ids, skip_special_tokens=True)
#    preds = [pred.strip() for pred in preds]
#   return preds


#predict_step(image_paths[:20])

In [6]:
#import matplotlib.pyplot as plt

#def show_image_with_caption(image_path, caption):
#    image = Image.open(image_path).convert('RGB')
    
#    plt.figure(figsize=(10, 5))
#    plt.imshow(image)
#    plt.title(caption, fontsize=15)
#    plt.axis('off')
#    plt.show()

# Example usage:
#n = 102
#image_path = image_paths[n]
#caption = predict_step([image_path])[0]
#show_image_with_caption(image_path, caption)

In [7]:
# Load the dataframe
#path = '/kaggle/input/memotion-dataset-7k/memotion_dataset_7k/labels.csv'
#df = pd.read_csv(path)

# Define a function to get image paths
#def get_image_path(image_name):
#    return f"{image_folder_path}/{image_name}"

#df['image_path'] = df['image_name'].apply(get_image_path)

# Batch processing with progress bar
#batch_size = 128
#captions = []

#for i in tqdm(range(0, len(df), batch_size), desc="Generating Captions"):
#    batch_paths = df['image_path'].iloc[i:i+batch_size].tolist()
#    batch_captions = predict_step(batch_paths)
#    captions.extend(batch_captions)

#df['caption'] = captions

# Save the updated dataframe
#df.to_csv('/kaggle/working/memotion_dataset_with_captions.csv', index=False)

# Show the updated dataframe
#df.head()

## BLIP image captioning

In [8]:
# Initialize the processor and model
processor = BlipProcessor.from_pretrained("Salesforce/blip-image-captioning-large")
model = BlipForConditionalGeneration.from_pretrained("Salesforce/blip-image-captioning-large").to("cuda")

preprocessor_config.json:   0%|          | 0.00/445 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/527 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/4.60k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.88G [00:00<?, ?B/s]

In [9]:
def predict_step(image_paths):
    images = []
    # Define generation parameters
    max_length = 512
    num_beams = 4
    gen_kwargs = {"max_length": max_length, "num_beams": num_beams}
    
    for image_path in image_paths:
        i_image = Image.open(image_path)
        if i_image.mode != "RGB":
            i_image = i_image.convert(mode="RGB")
        images.append(i_image)
    
    text = 'a picture of'
    inputs = processor(images=images, text=[text] * len(images), return_tensors="pt", padding=True).to("cuda")
    pixel_values = inputs["pixel_values"]
    input_ids = inputs["input_ids"]
    attention_mask = inputs["attention_mask"]

    #output_ids = model.generate(input_ids=input_ids, attention_mask=attention_mask, pixel_values=pixel_values, **gen_kwargs)
    
    output_ids = model.generate(pixel_values, **gen_kwargs)
    preds = processor.batch_decode(output_ids, skip_special_tokens=True)
    preds = [pred.strip() for pred in preds]
    return preds

In [10]:
#predict_step(image_paths[:20])

In [11]:
def show_image_with_caption(image_path, caption):
    image = Image.open(image_path).convert('RGB')
    
    plt.figure(figsize=(10, 5))
    plt.imshow(image)
    plt.title(caption, fontsize=15)
    plt.axis('off')
    plt.show()

In [12]:
#n = 767
#image_path = image_paths[n]
#caption = predict_step([image_path])[0]
#show_image_with_caption(image_path, caption)

In [13]:
import pandas as pd
from tqdm import tqdm

# Load the dataframe
path = '/kaggle/input/memotion-dataset-7k/memotion_dataset_7k/labels.csv'
df = pd.read_csv(path)

# Define a function to get image paths
def get_image_path(image_name):
    return f"{image_folder_path}/{image_name}"

df['image_path'] = df['image_name'].apply(get_image_path)

# Batch processing with progress bar
batch_size = 16
captions = []

for i in tqdm(range(0, len(df), batch_size), desc="Generating Captions"):
    batch_paths = df['image_path'].iloc[i:i+batch_size].tolist()
    batch_captions = predict_step(batch_paths)
    captions.extend(batch_captions)

df['caption'] = captions

# Save the updated dataframe
df.to_csv('/kaggle/working/memotion_dataset_with_captions.csv', index=False)

# Show the updated dataframe
df.head()

Generating Captions:  95%|█████████▌| 417/437 [1:37:55<03:57, 11.86s/it]/opt/conda/lib/python3.10/site-packages/PIL/Image.py:992: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(
Generating Captions: 100%|██████████| 437/437 [1:42:02<00:00, 14.01s/it]


,Unnamed: 0,image_name,text_ocr,text_corrected,humour,sarcasm,offensive,motivational,overall_sentiment,image_path,caption
0,0,image_1.jpg,LOOK THERE MY FRIEND LIGHTYEAR NOW ALL SOHALIK...,LOOK THERE MY FRIEND LIGHTYEAR NOW ALL SOHALIK...,hilarious,general,not_offensive,not_motivational,very_positive,/kaggle/input/memotion-dataset-7k/memotion_dat...,a couple of toy story characters standing next...
1,1,image_2.jpeg,The best of #10 YearChallenge! Completed in le...,The best of #10 YearChallenge! Completed in le...,not_funny,general,not_offensive,motivational,very_positive,/kaggle/input/memotion-dataset-7k/memotion_dat...,the best of 10 year challenge completed in les...
2,2,image_3.JPG,Sam Thorne @Strippin ( Follow Follow Saw every...,Sam Thorne @Strippin ( Follow Follow Saw every...,very_funny,not_sarcastic,not_offensive,not_motivational,positive,/kaggle/input/memotion-dataset-7k/memotion_dat...,there is a green trash can next to a green tra...
3,3,image_4.png,10 Year Challenge - Sweet Dee Edition,10 Year Challenge - Sweet Dee Edition,very_funny,twisted_meaning,very_offensive,motivational,positive,/kaggle/input/memotion-dataset-7k/memotion_dat...,there are two pictures of a woman and an ostri...
4,4,image_5.png,10 YEAR CHALLENGE WITH NO FILTER 47 Hilarious ...,10 YEAR CHALLENGE WITH NO FILTER 47 Hilarious ...,hilarious,very_twisted,very_offensive,not_motivational,neutral,/kaggle/input/memotion-dataset-7k/memotion_dat...,two pictures of a man and a woman with a micro...
